In [1]:
from pathlib import Path
import pandas as pda
import sys
sys.path.append('../')
from src.DataHandling import DataHandling
%load_ext autoreload
%autoreload 2

In [2]:
project_folder_name = "../data/"
project_parameter_file = "parameter.yml"

dataHandler = DataHandling(project_folder_name, project_parameter_file)

Loading parameter file ... ../data/parameter.yml


In [3]:
raw_pdCommits = dataHandler.unpickle_From_Raw_Data_Folder("raw_pdCommits.p")
raw_pdUsers = dataHandler.unpickle_From_Raw_Data_Folder("raw_pdUsers.p")
raw_pdReleases = dataHandler.unpickle_From_Raw_Data_Folder("raw_pdReleases.p")
raw_pdEdits = dataHandler.unpickle_From_Raw_Data_Folder("raw_pdEdits.p")

## Check user list

In [4]:
github_users_black_list = ["github-classroom[bot]", "actions-user", "web-flow"]
github_users_black_list_ids = raw_pdUsers [raw_pdUsers.login.isin(github_users_black_list)].anonym_uuid.unique().tolist() 
github_users_black_list_ids

['would-hot-power-president',
 'say-will-likely-office',
 'can-expect-late-month']

In [5]:
raw_pdCommits['commiter_role'] = "student"
raw_pdCommits.loc[raw_pdCommits.author.isin(github_users_black_list_ids), "commiter_role"] = 'github'

In [6]:
raw_pdUsers[~raw_pdUsers.anonym_uuid.isin(github_users_black_list_ids)]

,anonym_uuid,id,login,name,team
2,reach-young-financial-family,MDQ6VXNlcjczMTg5NjQ5,fb89zila,None,buschgommlic
3,take-economic-new-hand,MDQ6VXNlcjczNTM0MTYz,MarkusGom,None,buschgommlic
7,change-might-legal-minute,MDQ6VXNlcjgwNDA2MTIx,JoRiBi,None,code4fu
9,buy-sure-room-art,MDQ6VXNlcjY2Nzc4NDkz,Mezekr,Mezekr Weldu,code4fu
12,talk-change-single-city,MDQ6VXNlcjYzMjI4NDM1,Speed-Rabbit14,Speed-Rabbit14,codemaste
14,bring-political-fact-time,MDQ6VXNlcjYzNDY4ODg5,Divino21,None,codemaste
17,like-buy-final-hand,MDQ6VXNlcjczMTI4NTY4,jh-488,None,erler_hyad
19,want-start-huge-reason,MDQ6VXNlcjczNTM1NzQ5,Silbersaxe,None,erler_hyad
22,fall-hard-strong-door,MDQ6VXNlcjgzMjExODkz,EliasTechnik,None,evsw
24,look-try-significant-company,MDQ6VXNlcjczMDc4MzM4,Spidervini,None,evsw


## Eliminate none authors

In [7]:
raw_pdCommits[raw_pdCommits.author.isnull()]

,commit_sha,commited_at,no_of_modifications,commit_message,parent_sha,in_main_branch,author,committer,tag,branch_ids,team,task,unknown_user,commiter_role
29,1309a1cb789834853361504e7ae09c4a1afa0a3d,2021-05-27 16:59:37,1,Erste Änderung der Datei CSharpBasics.txt in C...,459beb6a49f60a27365b743b799c4909fed08a83,1,None,None,,[0],codemaste,3,Divino,student
33,1f8e1ffd6234b7104aeae99e4bb1052ce92204a0,2021-05-27 17:31:45,1,Erste Änderung der Datei CSharpBasics.md,1fb269f78bb8e4460a492d922980899cd9002ea1,1,None,None,,[0],codemaste,3,Divino,student
34,751a7781ad01491f3358858b579d8f6c471d8730,2021-05-27 17:36:19,1,Erste Änderung der Datei CSharpBasics.md,1f8e1ffd6234b7104aeae99e4bb1052ce92204a0,1,None,None,,[0],codemaste,3,Divino,student
36,9d2997fcbb9a20353efbbfc1533610c07694fa27,2021-05-27 17:43:09,1,Erste Änderung der Datei CSharpBasics.md,751a7781ad01491f3358858b579d8f6c471d8730,1,None,None,,[0],codemaste,3,Divino,student
188,f9b88240d144cae1edf5f5ab88c17ad7ac5a4d3a,2021-05-20 16:47:39,3,Teamrollen eingetragen,4a32433a93854a1896fbbfe9db75af2d3ecf5b2e,1,None,None,,[0],team,3,robert.mende@chemie.tu-freiberg.de,student
190,ccb40f752052701fca8137ec3d11eb6549844d0a,2021-05-20 17:30:49,1,Gitignore hinzugefügt,7da8692d4986ff5be189063c2201d9ab0e28d71d,1,None,None,,[0],team,3,robert.mende@chemie.tu-freiberg.de,student


In [8]:
missing_authors = {'robert.mende@chemie.tu-freiberg.de': 'mean-grow-real-child',
                   'RaginRain': "look-spend-dead-side",
                   'Divino': 'bring-political-fact-time'}

missing_authors_names = []

for index, row in raw_pdCommits[raw_pdCommits.author.isnull()].iterrows():
    print(row.author, ", ", row.unknown_user, " --> ", missing_authors[row.unknown_user])
    raw_pdCommits.at[index, "author"]  =  missing_authors[row.unknown_user]
    raw_pdCommits.at[index, "committer"]  =  missing_authors[row.unknown_user]

None ,  Divino  -->  bring-political-fact-time
None ,  Divino  -->  bring-political-fact-time
None ,  Divino  -->  bring-political-fact-time
None ,  Divino  -->  bring-political-fact-time
None ,  robert.mende@chemie.tu-freiberg.de  -->  mean-grow-real-child
None ,  robert.mende@chemie.tu-freiberg.de  -->  mean-grow-real-child


## Anonymize authors and teams

In [9]:
authors = raw_pdCommits.author.unique()
authors_dict = dict(zip(authors, range(len(authors))))

teams = raw_pdCommits.team.unique()
teams_dict = dict(zip(teams, range(len(teams))))

In [10]:
def annoymizeDataFrame(raw_df, teams_dict, authors_dict):
    df = raw_df.copy()
    if 'author' in df.columns:
        df['authorKey'] = df['author'].apply(lambda s: authors_dict.get(s) if s in authors_dict else s)
    if 'team' in df.columns:
        df['teamKey'] = df['team'].apply(lambda s: teams_dict.get(s) if s in teams_dict else s)
    return df

anonym_pdCommits = annoymizeDataFrame(raw_pdCommits, teams_dict, authors_dict)
anonym_pdReleases = annoymizeDataFrame(raw_pdReleases, teams_dict, authors_dict)
anonym_pdEdits = annoymizeDataFrame(raw_pdEdits, teams_dict, authors_dict)

## Save files

In [11]:
dataHandler.pickle_File_To_Preproc_Data_Folder(anonym_pdCommits, "anonym_pdCommits.p")
dataHandler.pickle_File_To_Preproc_Data_Folder(anonym_pdReleases, "anonym_pdReleases.p")
dataHandler.pickle_File_To_Preproc_Data_Folder(anonym_pdEdits, "anonym_pdEdits.p")